In [38]:
import pandas as pd
import seaborn as sns
#import pandas_profiling
import numpy as np
df = sns.load_dataset('iris')

### データの構造把握

In [ ]:
# 全列の（データ数、nullの有無、データ型）を表示
df.info()
# データの列数、行数を表示
df.shape
# データ型を表示
df.dtypes
# カラム一覧を表示
df.columns
# 各列のユニーク値の種類数を表示
df.apply(lambda x: x.nunique())
# 各列の欠損値の確認
df.isnull().sum()
# 指定したオブジェクトデータ列の項目別の数量を表示(NaN含む)
df['col_1'].value_counts(dropna=False)

In [31]:
# グループバイによる集計
df.groupby('species').mean()

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


In [ ]:
# ピボットテーブルによる集計
'''
index: 縦軸
columns: 横軸
values: 値
aggfunc: 集計方法
※index, columns, valuesには元データの列名のリストを指定することも可能
'''
df_pivot = df.pivot_table(index='date', columns='shash_nam', values='production_count', aggfunc='sum')

### データクリーニング

#### 欠損データ処理

In [39]:
# 欠損値を含む行を削除
df = df.dropna()
# 欠損値を含む列を削除
df = df.dropna(axis=1)
# 指定した列の欠損値が含まれている行だけを削除
df = df.dropna(subset=['列名'])

# 欠損値を任意の値で埋める
## 0で埋める場合
df = df.fillna(0)
## 文字で埋める場合
df = df.fillna('任意の文字列')
## 列を選んで適用することも可能
df['列名'] = df['列名'].fillna(0)
## 列ごとに指定の値で欠損値を埋める
df = df.fillna({'列名1': 0, '列名2': 5000, '列名3': 35})

#### 重複排除

In [ ]:
# デフォルトはkeep='first'(最初の行が残る)※lastにすると最後の行が残る
df.drop_duplicates()

# 重複を判定するカラムの指定
## 書き方1
df.drop_duplicates(subset='state')
df.drop_duplicates(subset=['state', 'point'])
## 書き方2 
df.drop_duplicates(['col_1'])
df.drop_duplicates(['col_1'], ['col_2'])

# 
df.drop_duplicates(subset='state', keep='last', inplace=True)

#### 文字列操作

In [ ]:
# 文字列置換
## 文字列に含まれる特定の文字を変換
series.str.replace(' ', '_')
## 小文字変換
s_new = s.str.lower()
## 大文字変換
s_new = s.str.upper()

# 文字列の削除
## 左右（先頭・末尾）両方の空白を削除する
s_new = s.str.strip()
## 引数に除去する文字を指定
s_new = s.str.strip('x')
## 先頭の空白を削除
s_new = s.str.lstrip()
## 末尾の空白を削除
s_new = s.str.rstrip()

# 文字列の部分抽出
## 先頭2文字だけ抽出
n1 = df['name'].str[:2]
## 末尾3文字の抽出
n2 = df['name'].str[-3:]
## 先頭2文字目から3文字目までを抽出（2文字分）
n5 = df['name'].str[1:3]

#### 日付系処理

In [ ]:
# 基本: 読み込み時に変換忘れたときとか
df['timestamp'] = pd.to_datetime(df['timestamp'])

#### 置換処理

In [ ]:
# 要素の置換
df.replace('CA', 'California')
df.replace(24, 100)
# 複数の異なる要素を一括で置換
df.replace({'CA': 'California', 24: 100})
df.replace(['CA', 24], ['California', 100]) # 第一引数に元の値のリスト、第二引数に置換後の値のリスト

### データフレーム操作

#### loc,iloc

In [ ]:
# loc,ilocによる操作
## 1行目のカラム2にアクセス
df.loc['row_1', 'col_2']
df.iloc[1, 2]
## データの取得だけではなく、代入も可能
df.loc['row_1', 'col_2'] = '0'
print(df.loc['row_1', 'col_2'])
df.iloc[1, 2] = '12'
print(df.iloc[1, 2])
## スライスでの範囲指定
df.iloc[3:5, 0:2]
df.loc[:, ['col_1', 'col_2']] # 行の指定を:(全体のスライス)にすることで、列を指定

#### 連結,結合

In [ ]:
# concat
## 基本 (縦に積む: カラムは各DataFrameの和集合
df = pd.concat([df_1, df_2, df_3])

## 横につなげる
df = pd.concat([df_1, df_2], axis=1)

In [ ]:
# 基本 (内部結合)
df = pd.merge(df, df_sub, on='key')

# 複数のカラムをキーとする
df = pd.merge(df, df_sub, on=['key_1', 'key_2'])

# 左結合
df = pd.merge(df, df_sub, on='key', how='left')

# 左右でカラム名が違うとき
df = pd.merge(df, df_sub,
              left_on='key_left', right_on='key_right') \
       .drop('key_left', axis=1)  # キーが両方残るのでどちらか消す

#### データ抽出

In [ ]:
# 指定した列をまるごと削除
df = df.drop("列名", axis=1)

# 基本
df[df.age >= 25]

# OR条件
df[(df.age <= 19) | (df.age >= 30)]

# AND条件
df[(df.age >= 25) & (df.age <= 34)]

## betweenでも書ける (あまり見ないが)
df[df['age'].between(25, 34)]

# query記法
new = df.query("Pclass==1 and Age >= 30")

# 日付でフィルタ
df['20130102':'20130104']

# データ型による列選択
## 特定の型を含む列の抽出
df.select_dtypes(include=['object'])
df.select_dtypes(include=['number', 'bool'])
## 特定の型を含まない列の抽出
df.select_dtypes(exclude=['object'])

#### filterを使った高度な抽出

In [ ]:
# 'apple', 'pineapple'が含まれる行の抽出
df.filter(items=['apple', 'pineapple'], axis=0)
# 'A', 'C'が含まれる列の抽出
df.filter(items=['A', 'C'], axis=1)
# 行、列を同時に条件指定して抽出
df.filter(items=['A', 'C']).filter(items=['apple', 'pineapple'], axis=0)
# 行名・列名が部分一致する（〜を含む）行・列を抽出するには、引数likeに文字列を指定する
df.filter(like='apple', axis=0)
# 行名・列名を正規表現で選択して行・列を抽出するには、引数regexに正規表現パターンを文字列で指定する。
df.filter(regex='e$', axis=0)

#### その他

In [ ]:
# 100行抽出
df.sample(n=100)

# 25%抽出
df.sample(frac=0.25)

# seed固定
df.sample(frac=0.25, random_state=42)

# 重複許可: デフォルトはreplace=False
df.sample(frac=0.25, replace=True)

# 列をサンプリング
df.sample(frac=0.25, axis=1)

# ソート
## 基本
df.sort_values(by='col_1')
## キーを複数 & 降昇順指定
'''
デフォルトは昇順(小さい順)。
降順にするには引数ascendingをFalseにする
'''
df.sort_values(by=['col_1', 'col_2'], ascending=[False, True])

### 新たなカラムの追加

#### if文ベース

In [ ]:
## 町名がSwampscottの行は1,それ以外は0のカラムの追加
df["FLG2"] = np.where(df["TOWN"] == "Swampscott", 1, 0)
## RM>=7.5の場合、大、それ以外中小のカラムの追加
df["FLG2_2"] = np.where(df["RM"] >= 7.5, "大", "中小")
## 複合条件
df["FLG2_3"] = np.where((df["RM"] >= 7.5) & (df["RM"] <= 8.0), 1, 0)

#### if-elif-elseベース

In [ ]:
def set_rm_kbn(row):
    if ((row.TOWN == 'Swampscott') & (row.RM >= 8)):
        return "大"
    elif ((row.TOWN == 'Swampscott') & (row.RM >= 6) & (row.RM < 8)):
        return "Swampscott小中"
    elif (row.RM < 8):
        return "Unknown小中"
    else:
        return "OTHERS"

df["FLG3"] = df.apply(set_rm_kbn, axis = 1)